In [ ]:
# If you use Google Colab, uncomment these lines:
#! wget https://raw.githubusercontent.com/Ayagoz/Tips-Tricks/master/lesson_1/utils.py
#! wget https://raw.githubusercontent.com/Ayagoz/Tips-Tricks/master/lesson_1/dataset.py
#! wget https://raw.githubusercontent.com/Ayagoz/Tips-Tricks/master/lesson_1/adult.csv

In [ ]:
! pip install category-encoders

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np 

from functools import partial

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

from dataset import Dataset
from utils import Compose, ScalerTransform

import  matplotlib.pyplot as plt
%matplotlib inline

Категориальные данные --  качественно характеризующие исследуемый процесс или объект, не имеющие количественного выражения.

Категориальный признак имеет N категорий.

Способы представления: 
    1. label-encoding:
        Метод label-encoding кодирует N признаков в числа от 0 до N-1.
        Пример: N категорий = {'march', 'feb', 'april'}
        тогда признак:
                'march' -> 0
                'april' -> 2
                'april' -> 2
                'feb'   -> 1
        Какой явный недостаток у такого метода?
        Ответ: ...
    2. one-hot-encoding:
        Метод one-hot-encoding кодирует признаки в вектора длины N, где 1 стоит на месте соответствующего значения.
        Пример: N категорий = {'march', 'feb', 'april'}
        тогда признак:
            'march' -> $[1, 0, 0]$
            'april' -> $[0, 0, 1]$
            'april' -> $[0, 0, 1]$
            'feb'   -> $[0, 1, 0]$
    3. binary-encoding:
        Метод binary-encoding сначала кодирует признаки как label-encoding, дальше преобразует числа от 0 до N-1 
        в двоичный код, далее двоичный код преобразуется в вектор.
        от 0 до N-1 = 7, 3 -> '011' -> $[0, 1, 1]$    
        Чем метод one-hot-encoding хуже чем binary-encoding?
        Ответ: ... 
    4. backward-difference-encoding:
        Признак состоящий из N категорий, или уровней, обычно входит в регрессию как последовательность фиктивных 
        переменных N-1.
        Почитать про это можно по ссылке ниже:
[constrast coding](https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/#backward)        
И еще существует какое-то количество других методов, но пока ограничимся этими.

Задание: Предсказать получает ли за год больше 50К по данным `adult.csv`. 
Добиться уровня качества (accuracy) >= 0.851.

Некоторые трюки:
    1. Есть несколько способ преобразовать категориальные признаки, попробуйте разные. 
    2. Иногда работает преобразовать некоторые признаки нелиненым способом, например взять логарифм
    или завести квадраты переменных. (это называется feature engineering)
    3. Отбор переменных. В случае когда есть много скоррелированных признаков -- это может ухудшать обучения ака
    проклятие размерности. (это называется feature selection)
    4. У модели есть параметры, которые можно менять. Это может очень влиять на качество. 

Описание переменных:

* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 
    1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, 
    Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, 
    Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, 
    Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, 
    Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, 
    Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
# С помощью трансформов вы можете задавать преобразования датасета для feature engineering

transforms = Compose([ScalerTransform(),
                      
                     ])
# Параметр cat_preproc_type - отвечает за разные способы преобразования категориальных признаков
dataset = Dataset("./adult.csv", target='salary',
                  cat_preproc_type='backward', drop=None,
                  transforms=transforms)

X, y = dataset.get_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
X_train.shape

In [ ]:
def get_accuracy(X_train, y_train, X_test, y_test, params, algo=LogisticRegression):
    model = algo(**params)
    # here tour code to fit model and measure predictions quality
    return acc

### Ваш бейзлайн

In [ ]:
get_accuracy(X_train, y_train, X_test, y_test, params={}, algo=BernoulliNB)

In [ ]:
def get_features(X, idx):
    return X[np.ix_(np.arange(len(X)), idx)].copy()

In [ ]:
def feature_selection(X_train, y_train, X_test, y_test, score_func, eps=1e-3):
    _, n = X_train.shape
    idx = []
    acc = [0]
    all_idx = np.arange(n)
    while len(idx) < n:
        best_acc = 0
        best_idx = None
        for i in all_idx:
            #here your code to find best params
            
        if best_acc - acc[-1] > eps:
            idx.append(best_idx)
            acc.append(best_acc)
            print(f'New acc {acc[-1]}')
        else:
            break

        
    return idx, acc

In [ ]:
params = {}
score_func = partial(get_accuracy, params=params, algo=BernoulliNB)

idx, acc = feature_selection(X_train, y_train, X_test, y_test, score_func, eps=1e-12)

In [ ]:
plt.plot(np.arange(len(idx)), acc[1:], 'go--', markersize=12, label='accuracy');
plt.xticks(ticks=np.arange(len(idx)), labels=idx);
plt.xlabel('Feature index');
plt.ylabel('Accuracy score');